# Plotting a Skew-T from Local Launch Data

This notebook demonstrates reading in one of our launch file datasets and plots the sounding using MetPy.

In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt
import metpy.calc as mpcalc
from metpy.plots import SkewT
from metpy.units import units, pandas_dataframe_to_unit_arrays
import numpy as np
from siphon.simplewebservice.wyoming import WyomingUpperAir

### Reading Local Data

Our radiosonde launch system provides data in a number of formats. At the top of this notebook there is a function that is set up to ready the SHARPPY ready file format that is available on our sounding page at http://bergeron.valpo.edu/soundings/. To use this data, select the appropriate sounding from the drop down menus near the top of the page, then click on the "SharpPy Ready File", this will open a new window, copy and past the full web address into the read function below.

In [ ]:
def read_SHARPPY(filename):
    import pandas as pd

    df = pd.read_csv(filename, skiprows=6, skipfooter=1,
                     names=['pressure','height','temperature','dewpoint','direction','speed'],
                     engine='python')
    df['u_wind'], df['v_wind'] = mpcalc.wind_components(df['speed'].values * units.knots,
                                                        df['direction'].values * units.degrees)

    df.units = dict(pressure='hPa', height='meter', temperature='degC', dewpoint='degC',
                    direction='degrees', speed='knot', u_wind='knot', v_wind='knot')
    return df

### Process Data

In [ ]:
# Set station and date/time for plotting purposes
station = 'VUM'
date = datetime(2019, 3, 5, 10, 57)
launch_number = 1

# Get back similar dataframe to what you get from the Wyoming archive
df2 = read_SHARPPY('http://bergeron.valpo.edu/soundings/launches/'
                   f'{date:%Y%m%d}_{launch_number:02d}/{date:%Y%m%d}_{launch_number:02d}_SHARPPY.txt')

# Get united arrays from our dataframe
data2 = pandas_dataframe_to_unit_arrays(df2)

# Isolate united arrays for calculation and plotting
p = data2['pressure']
T = data2['temperature']
Td = data2['dewpoint']
u = data2['u_wind']
v = data2['v_wind']

### Make the skew-T Plot

In [ ]:
fig = plt.figure(figsize=(10, 10))

skew = SkewT(fig, rotation=45)

# Plot the data using normal plotting functions, in this case using
# log scaling in Y, as dictated by the typical meteorological plot
skew.plot(p, T, 'r')
skew.plot(p, Td, 'g')

# Create a slice variable to help reduce the total
# number of wind barbs
wind_slice = slice(None, None, 15)
skew.plot_barbs(p[wind_slice], u[wind_slice], v[wind_slice], y_clip_radius=0.03)

# Set sensible axis limits
skew.ax.set_ylim(1000, 100)
skew.ax.set_xlim(-40, 30)

# Add the relevant special lines
skew.plot_dry_adiabats(t0=np.arange(233,555,10)*units.K)
skew.plot_moist_adiabats(color='tab:green')
skew.plot_mixing_lines(color='tab:blue')

# Plot some titles
plt.title('Station: K{}'.format(station), loc='left')
plt.title('Skew-T/Log-p', loc='center')
plt.title('{} UTC'.format(date), loc='right')

# Show the plot
#plt.savefig('KVUM_20190305_1057.png', bbox_inches='tight', dpi=150)
plt.show()